Please remember to update the token. For multiplayer to work, each player needs a different token. Please feel free to use multiple tokens in the same group.

In [ ]:
#Update your token
STUDENT_TOKEN = 'GROUP 1'

There should be no need to change the code below.

In [ ]:
## ignore this code, just used for submission
import requests
import pprint
import json
import random
import time
from copy import copy, deepcopy

class Game:
  def __init__(self, state, status, player):
    self.state = state
    self.status = status
    self.player = player

  def is_waiting(self):
    return self.status == 'waiting'

  def is_end(self):
    return self.status == 'complete'
  
  def get_winner(self):
    return None

  def actions(self):
    return []

  def print(self):
    print(self.state)

def new_game(game_type, multi_player = False):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/new-game?TOKEN=%s&game-type=%s&multi-player=%s' % (STUDENT_TOKEN, game_type, 'True' if multi_player else 'False'))
    if r.status_code == 200:
      return r.json()['game-id']
    print(r.content)

def join_game(game_type, game_id):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/join-game?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return r.json()['player']
    print(r.content)

def game_state(game_type, game_id, GameClass):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/game-state?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return GameClass(r.json()['state'], r.json()['status'], r.json()['player'])
    print(r.content)

def update_game(game_type, game_id, player, move):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/update-game?TOKEN=%s&game-type=%s&game-id=%s&player=%s&move=%s' % (STUDENT_TOKEN, game_type, game_id, player, move))
    if r.status_code == 200:
      return r.content
    print(r.content)

def game_loop(solver, GameClass, game_type, multi_player = False, id = None):
  while id == None:
    print('Creating new game...')
    id = new_game(game_type, multi_player)

  print('Joining game with id: %s' % id)
  player = join_game(game_type, id)

  print('Playing as %s' % player)

  game = game_state(game_type, id, GameClass)
  print('Waiting for the other player to join...')
  while game.is_waiting():
    time.sleep(10)
    game = game_state(game_type, id, GameClass)

  while True:
    game = game_state(game_type, id, GameClass)
    game.print_game()
    if game.is_end():
      if game.player == '-':
        print('Server says there is a draw')
      else:
        print('Server says your agent ' + ('WON' if game.player == player else 'LOST'))
      return
    if game.player == player:
      print('Making next move...')
      move = solver(game)
      print(move)
      update_result = update_game(game_type, id, player, json.dumps(move))
      print(update_result)
    else:
      time.sleep(2)

There is almost certainly no need to change this code, but if you must you can.

In [ ]:
from functools import reduce
from copy import copy, deepcopy
import json
import random

# Note: Game object will be provided by the server


class GinRummy(Game):
    def __init__(self, state, status, player):
        Game.__init__(self, state, status, player)

    def actions(self):
        return []  # this should return the possible actions

    def get_winner(self):
        return '.'  # this should return the actual winner

    def other_player(self):
        if self.player == 'O':
            return 'X'
        if self.player == 'X':
            return 'O'

    def letter_to_symbol(letter):
        if letter == 'S':
            return '\u2664'
        if letter == 'H':
            return '\u2665'
        if letter == 'D':
            return '\u2666'
        if letter == 'C':
            return '\u2667'
        return letter

    def print_game(self):
        if self.player == 'O':
            print('X: %i, \033[1mO: %i\033[0m' % tuple(self.state['score']))
        else:
            print('\033[1mX: %i\033[0m, O: %i' % tuple(self.state['score']))

        for card in self.get_hand():
            print('%i%s' %
                  (card[0], GinRummy.letter_to_symbol(card[1])), end=' ')
        print()

        if self.state['discard']:
            print('Discard: %i%s' % (
                self.state['discard'][0], GinRummy.letter_to_symbol(self.state['discard'][1])))
        else:
            print('Discard: empty')

    def get_hand(self):
        return self.state['hand']

    def get_hand_state(self):
        return self.state['hand_state']
    
    def get_discard(self):
        return self.state['discard']
    


Instead of changing this code for a random solver, write new code that also returns the next move. For example, create a new "amazing_solver".

Solvers need to return lists:
*   If the hand status is 'pick', it needes to be ['deck'] or ['discard'] 
*   If the hand status is 'discard', it needs to be the action (discard, knock, gin), and the value and suit of the card to discard (there is always a discard card). E.g. ['gin', 4, 'H']

In [ ]:
def random_solver(game):
  hand_state = game.get_hand_state()

  if hand_state == 'pick':
    return ['deck'] if random.randint(0, 1) == 0 else ['discard']
  else:
    hand = game.get_hand()
    card = hand[random.randint(0, len(hand) - 1)]
    return ['discard', card[0], card[1]]

**Documentation**

-Gin Rummy Rules

Set: Three or four cards of the same rank, for example ♥8♠8♦8 or ♥Q♠Q♦Q♣Q
Sequence: Three or more cards in sequence in the same suit, for example ♥A♥2♥3 or ♣6♣7♣8♣9♣10. 

Aces are always low, never high, and runs can't wrap around, so Q,K,A,2 would not be a legal run.
Meld: A word for both Sets and Sequences. You might for example have three melds, where two of them are sets and one is a run. Each card can only be part of one set or run, for example if you have an 8 you cannot count it both as part of 8,8,8 and 7,8,9.

Stock: A deck of facedown cards, in the middle of the table. Players draw one card from the stock in every round.

Discard pile: A pile of faceup cards, placed next to the stock. Players discard one card onto the pile in every round.

Deadwood: Any cards in your hand that are not part of a meld.

Knocking: Ending the round by putting a card face down on the discard pile.

Gin: When all 10 cards in your hand are parts of melds and you have no deadwood.<br>


**-What should the algorithm do?<br>**

We are using 3 member vaiables from game object(from parameter in function solver).
hand_state : 'pick' or 'discard'
	'pick' - It means the turn to pick a card from discard pile or stock.
	'discard' - It means the turn to discard one card from hand cards.
action : It is the return value in our solver function. [act, value, suite] (As you know).
hand : We can get cards in hand from this member variable.<br>

**So, what should we do?**

They are namely two points: make return values for 'pick' and 'discard'.<br>

**-Action for 'pick'**
Two choices: pick from stock or pick from discard.
We know only card value and suite from discard (faceup card).
So we can decide this card help us to make better meld cards then we choose it or otherwise choose unknown card from stock.
That's all for 'pick'<br>

**-Action for 'discard'**
Three options : ‘discard’, ‘knock’ or ‘gin’<br>

'gin' - It's clear to do 'gin'. When all cards from hand cards except one, form meld cards, simply do 'gin'
'knock' - Check if possible to do 'knock' (sum of deadwood cards are equal or less than 10), then do 'knock'
'discard' - If we cannot do 'gin' and 'knock', we should 'discard' by choosing one useless card from deadwood cards.
We do all these action via do_discard function<br>


**-Meld**
In this algorithm, the most important part is that function for getting meld cards.

**3 functions:**
get_better_melds, get_type_melds, get_num_melds
<br>
get_type_melds - Get sequence meld cards.<br>

get_num_melds - Get set meld cards.<br>

get_better_melds - Combine two of above functions and get better meld cards.
<br>
Depends on the status of meld cards we can decide which card to be discards or make faster way to make 'knock' or 'gin'



In [ ]:
import random
import numpy as np
from functools import cmp_to_key


left_cards = []         #   stockcards
discard_cards = []      #   discarded cards
hand_cards = []         #   cards in hand
meld_cards = []         #   meld cards in hand
deadwood_cards = []     #   deadwood cards in hand

HAND_CARD_COUNT = 10    #   hand_cards count 

#   Comparator for sorting cards by their number and type
def letter_cmp(a, b):
    if a[1] > b[1]:
        return -1
    elif a[1] == b[1]:
        if a[0] > b[0]:
            return 1
        else:
            return -1
    else:
        return 1
letter_cmp_key = cmp_to_key(letter_cmp)

def get_num_melds(cards, count):
    num_sorted_cards = cards.copy()
    num_sorted_cards.sort()                 #   sort cards in hands by num order
    num_melds = []
    card_num = 0                            #   variable for storing initial card num to be compared(number, it should be none card num not from 1-13)
    card_count = 0
    one_meld = []                           #   one pack of meld cards found on hand
    for card in num_sorted_cards:           #   find all packs of meld cards and add them to num_melds
        if card[0] != card_num:
            if card_count >= count:
                num_melds += one_meld
                one_meld.clear()
            card_count = 1
            card_num = card[0]
            one_meld.clear()
            one_meld.append(card)
        elif card[0] == card_num:
            card_count += 1
            one_meld.append(card)
    if card_count >= count:
        num_melds += one_meld               #   here, saved all possible meld cards in num_melds
    if len(num_melds) > 0:
        for card in num_melds:
            if card in num_sorted_cards : num_sorted_cards.remove(card)         #   remove meld cards from hand cards to get deadwood cards
    return num_melds,num_sorted_cards       #   return two of them (meld, deadwood cards)

#   Get Sequence meld card
#   cards - cards where to find meld cards  
#   count - it should be 3 (meld cards minimum count) 
#   returns - [Sequence meld cards], [deadwood cards]
def get_type_melds(cards, count):
    type_sorted_cards = cards.copy()
    type_sorted_cards.sort(key=letter_cmp_key)          #   sort cards in hands by type and num order
    type_melds = []
    card_type = 'X'                                     #   variable for storing initial card type to be compared(number, it should be none card num not from 'S','H','D','C')
    card_num = 0
    card_count = 0
    one_meld = []                                       #   one pack of meld cards found on hand
    for card in type_sorted_cards:                      #   find all packs of meld cards and add them to type_melds
        if card[1] != card_type or card[0] != card_num + 1:
            if card_count >= count:
                type_melds += one_meld
                one_meld.clear()
            card_count = 1
            card_type = card[1]
            card_num = card[0]
            one_meld.clear()
            one_meld.append(card)
        elif card[1] == card_type and card[0] == card_num + 1:
            card_count += 1
            card_num = card[0]
            one_meld.append(card)
            
    if card_count >= count:
        type_melds += one_meld                          #   here, saved all possible meld cards in type_melds
    if len(type_melds) > 0:
        for card in type_melds:
            if card in type_sorted_cards : type_sorted_cards.remove(card)       #   remove meld cards from hand cards to get deadwood cards
    return type_melds, type_sorted_cards                #   return two of them (meld, deadwood cards)

#   Among 2 types of melds, choose one which has bigger count of meld cards
#   Compare each types of melds cards(Set or Sequence), choose bigger one
def get_better_melds(cards, count):
    newcards = cards.copy()                     
    type_rets = get_type_melds(newcards, count)            # get sequence meld cards
    type_melds = type_rets[0]
    type_deadwood = type_rets[1]
    num_rets = get_num_melds(newcards,count)               # get set meld cards
    num_melds = num_rets[0]
    num_deadwood = num_rets[1]
    if len(num_melds) > len(type_melds):                # compare two of them return better ones
        return num_melds, num_deadwood
    else:
        return type_melds, type_deadwood
    
#   After pick, call this function to check whether knock or not 
#   Basically, decide knock when the sum of deadwood cards are less or equal to 10
def check_knock():
    sum = 0
    for card in deadwood_cards:
        if card[0] > 10 : val = 10
        else: val = card[0]
        sum += val
    if(sum <= 10) : return True
    else: return False
    
#   Performs actions for pick
def do_pick(discard_card):
    global meld_cards
    global deadwood_cards
    
    one = discard_card
    
    new_hand_cards = hand_cards.copy()
    new_hand_cards.append(one)                          #   make new hand cards (hand_cards + discard_card), and then get melds cards from it to check it better for us
    
    rets = get_better_melds(new_hand_cards, 3)
    new_meld = rets[0]
    new_deadwood = rets[1]
    
    if len(new_meld) > len(meld_cards):                 # if new_hand_cards generates better meld cards, we choose discard card.
        return['discard']
    else:
        return['deck']                                  # if not, we choose stock card

#   When dicard, this function performs dicard actions
#   Choose card to be dicarded and if gin is possible then choose gin
def do_discard():
    global meld_cards
    global deadwood_cards
    
    deadwood_cards.sort()

    if len(deadwood_cards) > 0:
        one = deadwood_cards[len(deadwood_cards) - 1]   #   choose biggest one from deadwood_cards as discard
    else: one = hand_cards[len(hand_cards) - 1]
    
    print(deadwood_cards)
    if one in hand_cards : hand_cards.remove(one)       #   remove card to be discarded from hand cards
    ret = ['discard']
    if(len(deadwood_cards) < 2): ret = ['gin']  # deadwood card's count == 1 do gin
    elif (check_knock()): ret = ['knock']       # check whether we can do knock, then do knock
    ret.append(one[0])
    ret.append(one[1])
    discard_cards.append(one)
    return ret                                  # return result (i.e ['discard',5, 'S'])



def amazing_solver(game):
    global meld_cards
    global deadwood_cards
    result = []
    hand_state = game.get_hand_state()
    hand_cards = game.get_hand()
    
    rets = get_better_melds(hand_cards, 3)          #   get melds cards and deadwood cards from hand cards
    meld_cards = rets[0]
    deadwood_cards = rets[1]
    
    if hand_state == 'pick':
        return do_pick(game.get_discard())          #   get_discard() returns discard card from discard pile, so we can decide action for 'pick' with this card
    else:
        return do_discard()        #    when action is 'discard', call the function to manage discard action



In [ ]:
game_loop(amazing_solver, GinRummy,'rummy', multi_player=False, id=None)

Creating new game...
Joining game with id: 8414
Playing as X
Waiting for the other player to join...
X: 0, O: 0
4♦ 2♤ 7♤ 5♥ 12♧ 1♦ 3♥ 5♤ 10♥ 3♧ 
Discard: 9♧
Making next move...
['deck']
b'Valid move'
X: 0, O: 0
4♦ 2♤ 7♤ 5♥ 12♧ 1♦ 3♥ 5♤ 10♥ 3♧ 11♤ 
Discard: 9♧
Making next move...
[[1, 'D'], [2, 'S'], [3, 'C'], [3, 'H'], [4, 'D'], [5, 'H'], [5, 'S'], [7, 'S'], [10, 'H'], [11, 'S'], [12, 'C']]
['discard', 12, 'C']
b'Valid move'
X: 0, O: 0
4♦ 2♤ 7♤ 5♥ 1♦ 3♥ 5♤ 10♥ 3♧ 11♤ 
Discard: 8♧
Making next move...
['deck']
b'Valid move'
X: 0, O: 0
4♦ 2♤ 7♤ 5♥ 1♦ 3♥ 5♤ 10♥ 3♧ 11♤ 7♦ 
Discard: 8♧
Making next move...
[[1, 'D'], [2, 'S'], [3, 'C'], [3, 'H'], [4, 'D'], [5, 'H'], [5, 'S'], [7, 'D'], [7, 'S'], [10, 'H'], [11, 'S']]
['discard', 11, 'S']
b'Valid move'
X: 0, O: 0
4♦ 2♤ 7♤ 5♥ 1♦ 3♥ 5♤ 10♥ 3♧ 7♦ 
Discard: 2♦
Making next move...
['deck']
b'Valid move'
X: 0, O: 0
4♦ 2♤ 7♤ 5♥ 1♦ 3♥ 5♤ 10♥ 3♧ 7♦ 5♧ 
Discard: 2♦
Making next move...
[[1, 'D'], [2, 'S'], [3, 'C'], [3, 'H'], [4, 'D'], [7, 'D'], [7, 'S']

In [ ]:
game_loop(amazing_solver, GinRummy,'rummy', multi_player=False, id=None)

Creating new game...
Joining game with id: 8417
Playing as X
Waiting for the other player to join...
X: 0, O: 0
6♧ 2♥ 12♤ 4♦ 1♥ 1♦ 10♤ 11♤ 8♤ 12♥ 
Discard: 10♥
Making next move...
['deck']
b'Valid move'
X: 0, O: 0
6♧ 2♥ 12♤ 4♦ 1♥ 1♦ 10♤ 11♤ 8♤ 12♥ 2♤ 
Discard: 10♥
Making next move...
[[1, 'D'], [1, 'H'], [2, 'H'], [2, 'S'], [4, 'D'], [6, 'C'], [8, 'S'], [12, 'H']]
['discard', 12, 'H']
b'Valid move'
X: 0, O: 0
6♧ 2♥ 12♤ 4♦ 1♥ 1♦ 10♤ 11♤ 8♤ 2♤ 
Discard: 5♧
Making next move...
['deck']
b'Valid move'
X: 0, O: 0
6♧ 2♥ 12♤ 4♦ 1♥ 1♦ 10♤ 11♤ 8♤ 2♤ 9♧ 
Discard: 5♧
Making next move...
[[1, 'D'], [1, 'H'], [2, 'H'], [2, 'S'], [4, 'D'], [6, 'C'], [8, 'S'], [9, 'C']]
['discard', 9, 'C']
b'Valid move'
X: 0, O: 0
6♧ 2♥ 12♤ 4♦ 1♥ 1♦ 10♤ 11♤ 8♤ 2♤ 
Discard: 5♤
Making next move...
['deck']
b'Valid move'
X: 0, O: 0
6♧ 2♥ 12♤ 4♦ 1♥ 1♦ 10♤ 11♤ 8♤ 2♤ 10♧ 
Discard: 5♤
Making next move...
[[1, 'D'], [1, 'H'], [2, 'H'], [2, 'S'], [4, 'D'], [6, 'C'], [8, 'S'], [10, 'C']]
['discard', 10, 'C']
b'Valid move'
X: 0,